In [9]:
import open3d as o3d
import numpy as np
import os

# Path to the directory with RGB-D images
folder = r"D:\prisha_manipal_sp\sp_rugved\desk_2\rgbd-scenes\desk\desk_2"

# Intrinsics (can be updated if you have actual camera intrinsics)
intrinsic = o3d.camera.PinholeCameraIntrinsic(
    o3d.camera.PinholeCameraIntrinsicParameters.PrimeSenseDefault
)

In [10]:
# Combine all point clouds
combined_pcd = o3d.geometry.PointCloud()

for i in range(40, 45):  # 40 to 44 inclusive
    color_path = os.path.join(folder, f"desk_2_{i}.png")
    depth_path = os.path.join(folder, f"desk_2_{i}_depth.png")

    # Load RGB-D image
    color = o3d.io.read_image(color_path)
    depth = o3d.io.read_image(depth_path)

    rgbd = o3d.geometry.RGBDImage.create_from_color_and_depth(
        color, depth, convert_rgb_to_intensity=False
    )

    # Create point cloud from RGB-D
    pcd = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd, intrinsic)

    # Flip coordinate system
    pcd.transform([[1, 0, 0, 0],
                   [0, -1, 0, 0],
                   [0, 0, -1, 0],
                   [0, 0, 0, 1]])

    # Append to the combined cloud
    combined_pcd += pcd.voxel_down_sample(voxel_size=0.005)

In [14]:
# Denoise final point cloud
combined_pcd, _ = combined_pcd.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)

# Visualize
o3d.visualization.draw_geometries([combined_pcd])